In [119]:
import winsound
import pygame
from bs4 import BeautifulSoup, NavigableString
import requests
from time import sleep
from random import randint
import math
from tqdm import tqdm
import pickle
import pandas as pd
from datetime import date
from datetime import datetime
import json

import pymysql                        # for getting data from a SQL database
from sqlalchemy import create_engine, text  # for establishing the connection and authentication
import zlib

from webscraper.scrap_modules import scraper_ds as mybib
from webscraper.scrap_modules import prep as prep

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### load scraped raw data

In [134]:

with open("webscraper/webscrap_data/soup_dict.pkl", "rb") as f:
    scraper_dict = pickle.load(f)
    
with open("webscraper/webscrap_data/keyword_dict.pkl", "rb") as f:
    keyword_dict = pickle.load(f)

In [127]:
len(scraper_dict)

20061

### transform scraped raw data into df 

In [157]:
df = prep.transform_scrap_data_to_df(scraper_dict)

100%|██████████| 20941/20941 [12:51<00:00, 27.15it/s] 


In [176]:
df.head(10)

,id,title,company,city,posting_date,job_description,seniority_level,employment_type,job_function,industries,scraping_date,url
0,3685669741,Software Engineer Fullstack (m/w/d),Atruvia AG,"Aschheim, Bavaria, Germany",1 day ago,VollzeitWir sind der Digitalisierungspartner d...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
1,3719620327,Data Analyst - Business Intelligence,Almedia,"Berlin, Berlin, Germany",5 days ago,Almedia helps companies grow by promoting thei...,Mid-Senior level,Full-time,Analyst,"Technology, Information and Internet",2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
2,3720365801,(Junior) Data Engineer (m/f/d),Sandbox Interactive,"Berlin, Berlin, Germany",1 day ago,YOUR GAMEBuild and maintain the current data i...,Mid-Senior level,Full-time,Information Technology,Computer Games,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
3,3672508925,Financial Data Analyst (m/f/d),Ultramarin,"Berlin, Berlin, Germany",2 months ago,About The PositionWe are looking for a top-tie...,Mid-Senior level,Full-time,Information Technology,Financial Services,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
4,3695466781,Business Intelligence Analyst (f/m/d),Les Lunes,"Berlin, Berlin, Germany",4 weeks ago,Your missionAs a Business Intelligence Analyst...,Mid-Senior level,Full-time,"Research, Analyst, and Information Technology",Retail Apparel and Fashion,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
5,3585738180,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,Orange Quarter are working with an exciting st...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
6,3585734564,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,Orange Quarter are currently looking for a Dat...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
7,3704916810,BI Analyst (f/m/d),Enter,"Berlin, Berlin, Germany",3 weeks ago,Build the future with Enter.Our mission is to ...,Mid-Senior level,Full-time,Information Technology,Architecture and Planning,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
8,3585734595,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,We are currently working with an emerging SaaS...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
9,3698396795,Senior Data Analyst,Klim,"Berlin, Berlin, Germany",1 month ago,"We are Klim, a Berlin-based clean-tech company...",Mid-Senior level,Full-time,Other,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...


In [174]:
df["title"].isna().sum()

206

### data cleaning - empty rows
Trying to fill none values doing a second request

In [175]:
new_df = prep.second_request_empty_values(df, max_sleeptime = )

100%|██████████| 206/206 [00:03<00:00, 62.09it/s] 


In [169]:
new_df['title'].isna().sum()

16

 Manual research for the id's which have still empty rows

In [168]:
empty_rows = new_df[new_df["title"].isna()==True]
empty_rows

,id,title,company,city,posting_date,job_description,seniority_level,employment_type,job_function,industries,scraping_date,url
20779,3715670180,None,None,None,None,None,None,None,None,None,2023-09-25,None
20784,3470370115,None,None,None,None,None,None,None,None,None,2023-09-25,None
20785,3713820046,None,None,None,None,None,None,None,None,None,2023-09-25,None
20786,3708677238,None,None,None,None,None,None,None,None,None,2023-09-25,None
20789,3702068466,None,None,None,None,None,None,None,None,None,2023-09-25,None
20790,3724887274,None,None,None,None,None,None,None,None,None,2023-09-25,None
20791,3718984435,None,None,None,None,None,None,None,None,None,2023-09-25,None
20792,3716462365,None,None,None,None,None,None,None,None,None,2023-09-25,None
20794,3720125985,None,None,None,None,None,None,None,None,None,2023-09-25,None
20796,3727603747,None,None,None,None,None,None,None,None,None,2023-09-25,None


In [145]:
empty_id_list = empty_rows['id'].tolist()

for id in empty_id_list:
    print(f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{id}")

https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3718984435
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3716462365
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3701555920
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3714883910
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3709369178
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3722015348
https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3702308160


In [195]:
def null_value_cleaner(df, rounds):
    null_counter = df['title'].isna().sum()
    sleeptime = 2000
    for i in range(rounds):
        if null_counter == 0:
            print(f"Finished in round {id}")
            return df
        else:
            df = prep.second_request_empty_values(df, max_sleeptime = sleeptime)
            null_counter = df['title'].isna().sum()
            print(f"Round{i}: {null_counter} empty values left")
            sleeptime += 500
    return df

In [197]:
cleaned_df = null_value_cleaner(df, 4)

100%|██████████| 206/206 [00:06<00:00, 30.50it/s]


Round0: 57 empty values left


100%|██████████| 57/57 [00:00<00:00, 57.42it/s]


Round1: 0 empty values left
Finished in round 3702308160


In [189]:
display(cleaned_df)

None

In [212]:
small_dict = {}
for key in keyword_dict:
    small_dict[key] = keyword_dict[key][0]

In [215]:
def search_keyword(x, dict):
    for key in dict:
        if x == dict:
            y = dict[item]
            return y
        else:
            y = 'unknown'
    return y

In [217]:
cleaned_df['keyword'] = cleaned_df['id'].map(small_dict)

In [219]:
prep.save_cleaned_df(cleaned_df)